In [1]:
# Import required modules
import cv2 as cv
import math
import time

In [2]:
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [
                                104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2),
                         (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

In [3]:
#Load pre-trained models
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

In [4]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
           '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

In [5]:
# Load network
ageNet = cv.dnn.readNetFromCaffe(ageProto, ageModel)
genderNet = cv.dnn.readNetFromCaffe(genderProto, genderModel)
faceNet = cv.dnn.readNet(faceModel, faceProto)

In [8]:
# Open a camera stream or video file
cap = cv.VideoCapture(0) # Write video path instead of 0 to input video file
padding = 20
k=-cv.waitKey(1)
while cv.waitKey(1)<0:
    k = cv.waitKey(100)
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break
    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0, bbox[1]-padding):min(bbox[3]+padding, frame.shape[0]-1),
                     max(0, bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(
            face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]

        print("Gender : {}, confidence = {:.3f}".format(
            gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]

        print("Age : {}, confidence = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0]-5, bbox[1]-10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender Demo", frameFace)
        cv.waitKey(100)
    if cv.getWindowProperty('Age Gender Demo',0)<0:
        cv.destroyAllWindows()
        break
    else:
        pass
    print("Time : {:.3f}".format(time.time() - t))
cap.release()

Gender : Male, confidence = 0.965
Age : (25-32), confidence = 0.735
Time : 0.483
Gender : Male, confidence = 0.986
Age : (0-2), confidence = 0.885
Time : 0.221
Gender : Male, confidence = 0.840
Age : (0-2), confidence = 0.972
Time : 0.203
Gender : Male, confidence = 0.963
Age : (0-2), confidence = 0.911
Time : 0.205
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.959
Time : 0.204
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.965
Time : 0.204
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.641
Time : 0.204
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.866
Time : 0.203
Gender : Male, confidence = 0.999
Age : (25-32), confidence = 0.998
Time : 0.222
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.985
Time : 0.216
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.996
Time : 0.217
Gender : Male, confidence = 1.000
Age : (25-32), confidence = 0.990
Time : 0.205
Gender : Male, confidence = 1.000


In [7]:
#For image
padding = 20
frame = cv.imread('C:\\Users\\User\\Desktop\\Python\\Project8-Gender_Age_AI\\Project8-Gender_Age_AI\\1.jpg')
frameFace, bboxes = getFaceBox(faceNet, frame)
if not bboxes:
   print("No face Detected, Checking next frame")
   #continue

for bbox in bboxes:
   # print(bbox)
   face = frame[max(0, bbox[1]-padding):min(bbox[3]+padding, frame.shape[0]-1),
                max(0, bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
   blob = cv.dnn.blobFromImage(
            face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
   genderNet.setInput(blob)
   genderPreds = genderNet.forward()
   gender = genderList[genderPreds[0].argmax()]

   print("Gender : {}, confidence = {:.3f}".format(
       gender, genderPreds[0].max()))

   ageNet.setInput(blob)
   agePreds = ageNet.forward()
   age = ageList[agePreds[0].argmax()]

   print("Age : {}, confidence = {:.3f}".format(age, agePreds[0].max()))

   label = "{},{}".format(gender, age)
   cv.putText(frameFace, label, (bbox[0]-5, bbox[1]-10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv.LINE_AA)
   cv.imshow("Age Gender Demo", frameFace)
   cv.waitKey(0)

Gender : Male, confidence = 1.000
Age : (48-53), confidence = 0.463
